In [4]:
import requests
import json
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


In [16]:
import json
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

# Load settings
with open('setting.json', 'r') as data_file:
    sv = json.load(data_file)

# Login details and URLs
LOGIN_URL = "https://members.junglescout.com/users/sign_in.json"
NICHE_URL = "https://members.junglescout.com/#/niche"

login_data = {
    "user": {
        "login": "geekideaonline@gmail.com",
        "password": "Trockmanimes#132"
    }
}

def get_token(login_data):
    req = urllib.request.Request(LOGIN_URL)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    
    json_data = json.dumps(login_data)
    json_data_as_bytes = json_data.encode('utf-8')
    
    req.add_header('Content-Length', len(json_data_as_bytes))
    response = urllib.request.urlopen(req, json_data_as_bytes)
    
    bsObj0 = BeautifulSoup(response, "lxml")
    token_data = bsObj0.find("p").get_text()[50:82]
    return token_data

def get_product_ids(token):
    GETID_URL = "http://members.junglescout.com/api/v1/keywords/get_keywords?query=towel&direction=desc&column=opportunity_score&per_page=200&page=2&initialPageLoad=false&averagePriceMin=0&averagePriceMax=100&competitionMin=0&competitionMax=100&demandMin=0&demandMax=1000&listingQualityScoreMin=0&listingQualityScoreMax=100&opportunityScoreMin=0&opportunityScoreMax=100&wordCountMin=1&wordCountMax=4&country=us&categories=%5B%22Baby%22%5D"
    req = urllib.request.Request(GETID_URL)
    req.add_header('token', token)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    req.add_header('Host', 'members.junglescout.com')
    req.add_header('Connection', 'keep-alive')

    response = urllib.request.urlopen(req)
    bsObj = BeautifulSoup(response, "lxml")
    
    p_tag = bsObj.find("p").get_text()
    data = json.loads(p_tag)
    keywords = data["data"]["keywords"]
    
    return [item["id"] for item in keywords]

def get_product_details(token, product_id):
    PRODUCT_URL = f"http://members.junglescout.com/api/v1/keywords/keyword_data?id={product_id}"
    req = urllib.request.Request(PRODUCT_URL)
    req.add_header('token', token)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    req.add_header('Host', 'members.junglescout.com')
    req.add_header('Connection', 'keep-alive')

    response = urllib.request.urlopen(req)
    bsObj = BeautifulSoup(response, "lxml")

    p_tag = bsObj.find("p").get_text()
    data = json.loads(p_tag)
    products = data["data"]["products"]
    
    return products

def main():
    token = get_token(login_data)
    
    product_ids = get_product_ids(token)
    all_products = [get_product_details(token, product_id) for product_id in product_ids]
    
    df_prd = pd.DataFrame(columns=('1.Product Name', '2.Product Link', '3.Review', '4.Price', '5.Rank', '6.EST Sales', '7.Revenue'))
    for products in all_products:
        for product in products:
            item_detail = {
                "1.Product Name": product["product_name"],
                "2.Product Link": product["product_url"],
                "3.Review": product["reviews"],
                "4.Price": product["price"],
                "5.Rank": product["pd_rank"],
                "6.Weight": product["weight"],
                "7.EST Sales": product["pd_est_sales"],
                "8.Revenue": product["calc_estimated_revenue"]
            }
            df_prd = df_prd.append(item_detail, ignore_index=True)
    print(df_prd)
    try:
        response = urllib.request.urlopen(req, json_data_as_bytes)
    except urllib.error.HTTPError as e:
        print(f"HTTPError: {e.code} - {e.reason}")
    except urllib.error.URLError as e:
        print(f"URLError: {e.reason}")


if __name__ == "__main__":
    main()

HTTPError: HTTP Error 403: Forbidden